In [ ]:
# Install necessary libraries
!pip install kaggle
!pip install mysqlclient
!pip install pyodbc

# Import libraries
import kaggle
import zipfile
import pandas as pd
import sqlalchemy as sal
import pyodbc

# Downloading data files directly from Kaggle
# The file will be downloaded as a zip file
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

# Extracting the zip file
zip_ref = zipfile.ZipFile('orders.csv.zip', 'r')
zip_ref.extractall()
zip_ref.close()

# Reading the data
df = pd.read_csv('C:\\Users\\riyab\\Downloads\\orders.csv\\orders.csv')

# Show the first 20 rows of the data
df.head(20)

# Data cleaning and transformation
# Normalize column names
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

# Create new column 'discount'
df['discount'] = df['list_price'] * df['discount_percent'] * 0.01

# Create new column 'sale_price'
df['sale_price'] = df['list_price'] - df['discount']

# Create new column 'profit'
df['profit'] = df['sale_price'] - df['cost_price']

# Show the first 5 rows of the modified dataframe
df.head(5)

# Drop unnecessary columns
df.drop(columns=['list_price', 'cost_price', 'discount_percent'], inplace=True)

# Convert the 'order_date' column to datetime format
df['order_date'] = pd.to_datetime(df['order_date'], format="%Y-%m-%d")

# Connect to SQL Server and load data into SQL
engine = sal.create_engine('mssql+pyodbc://Riyyaaaaaa\\SQLEXPRESS/master?driver=ODBC+Driver+17+for+SQL+Server')

# Load the data into the SQL table 'df_orders'
conn = engine.connect()
df.to_sql('df_orders', con=conn, index=False, if_exists='append')

# Close the connection
conn.close()
